# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Sub problem 1
ident = orders.groupby(['CustomerID']).agg({'amount_spent':sum})
ident.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [4]:
#Sub problem 2
#I'm going to separate the clients by ranges, first I'm going to calculate the percentiles
#I got the function here: https://qastack.mx/programming/2374640/how-do-i-calculate-percentiles-with-python-numpy
a = np.array(ident['amount_spent'])
p75 = np.percentile(a, 75)
p95 = np.percentile(a, 95)

In [5]:
labels = ["Normal", "Preferred", "VIP"]
separators = [ident['amount_spent'].min(), p75, p95, ident['amount_spent'].max()]
bins = pd.cut(ident['amount_spent'], separators, labels = labels)
bins.value_counts()

Normal       3253
Preferred     868
VIP           217
Name: amount_spent, dtype: int64

In [6]:
#Sub problem 3
ident['Client_Type'] = pd.cut(ident['amount_spent'], separators, labels = labels)
ident.head()

,amount_spent,Client_Type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,Preferred
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# To get this result I'm going to use a Join function
join_table = pd.merge(left = ident,
                      right = orders,
                      how = 'inner', 
                      left_on = "CustomerID", 
                      right_on= "CustomerID")
display(join_table.shape)
join_table

(397924, 16)

,CustomerID,amount_spent_x,Client_Type,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent_y
0,12346,77183.60,VIP,61619,541431,23166,2011,1,2,10,medium ceramic top storage jar,74215,2011-01-18 10:01:00,1.04,United Kingdom,77183.60
1,12347,4310.00,Preferred,14938,537626,85116,2010,12,2,14,black candelabra t-light holder,12,2010-12-07 14:57:00,2.10,Iceland,25.20
2,12347,4310.00,Preferred,14939,537626,22375,2010,12,2,14,airline bag vintage jet set brown,4,2010-12-07 14:57:00,4.25,Iceland,17.00
3,12347,4310.00,Preferred,14940,537626,71477,2010,12,2,14,colour glass. star t-light holder,12,2010-12-07 14:57:00,3.25,Iceland,39.00
4,12347,4310.00,Preferred,14941,537626,22492,2010,12,2,14,mini paint set vintage,36,2010-12-07 14:57:00,0.65,Iceland,23.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,18287,1837.28,Preferred,392752,570715,22419,2011,10,3,10,lipstick pen red,12,2011-10-12 10:23:00,0.42,United Kingdom,5.04
397920,18287,1837.28,Preferred,392753,570715,22866,2011,10,3,10,hand warmer scotty dog design,12,2011-10-12 10:23:00,2.10,United Kingdom,25.20
397921,18287,1837.28,Preferred,423939,573167,23264,2011,10,5,9,set of 3 wooden sleigh decorations,36,2011-10-28 09:29:00,1.25,United Kingdom,45.00
397922,18287,1837.28,Preferred,423940,573167,21824,2011,10,5,9,painted metal star with holly bells,48,2011-10-28 09:29:00,0.39,United Kingdom,18.72


In [9]:
#I will take the 3 columns I need and I'll drop the duplicates
join_table=join_table[['CustomerID', 'Client_Type', 'Country']].drop_duplicates()
join_table

,CustomerID,Client_Type,Country
0,12346,VIP,United Kingdom
1,12347,Preferred,Iceland
183,12348,Preferred,Finland
214,12349,Preferred,Italy
287,12350,Normal,Norway
...,...,...,...
397069,18280,Normal,United Kingdom
397079,18281,Normal,United Kingdom
397086,18282,Normal,United Kingdom
397098,18283,Preferred,United Kingdom


In [10]:
count = join_table.groupby(['Client_Type', 'Country']).agg('count').sort_values('CustomerID', ascending=False)
count.head(3)

,,CustomerID
Client_Type,Country,
Normal,United Kingdom,2988
Preferred,United Kingdom,755
VIP,United Kingdom,177


In [ ]:
#As I can see the Country wich more VIP customers is United Kingdom

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [ ]:
#As I could observe in the last question, the Country wich more VIP 
#and preferred coustomers is United Kingdom